# 🎵 Phantom Track Generator [DARK WEB EDITION]

## 👇 下の「▶実行」ボタンをクリックするだけで使い始められます！

## 最適なランタイムタイプの設定方法（重要！）：
1. 上部メニューの「ランタイム」→「ランタイムのタイプを変更」をクリック
2. 「ハードウェアアクセラレータ」で「**GPU**」を選択（これが最強の設定です！）
3. 「保存」をクリック
4. その後、下の▶ボタンをクリック

## このアプリでできること：
- 既存の音楽を元にして、AIが新しい音楽を自動生成
- 好きな曲を最大20曲まで組み合わせて参考にできる
- 生成する音楽の雰囲気を日本語で指示できる
- 生成時間やジャンルなどを自由に設定できる

## 使い方（たった1クリック）：
1. **下にある▶ボタンをクリック**して実行（一番下のコードセルの左側にある実行ボタン）
2. 5〜10分待つとアプリが起動（初回は時間がかかります）
3. 表示されるURLをクリックしてアプリを開く
4. アプリ画面で音楽ファイルをアップロードして「INITIATE GENERATION」ボタンをクリック

## 詳しい使い方：
1. 「リファレンストラックファイル」に好きな曲（MP3/WAVなど）をアップロード（最大20曲）
2. 「生成指示コード」に「80年代シティポップ風の明るい曲」などと入力
3. 必要に応じて「音楽カテゴリ分類」や「高度パラメータ設定」を調整
4. 「INITIATE GENERATION」ボタンをクリック
5. 生成された曲を再生・ダウンロード

## 注意事項：
- アプリの起動には5〜10分程度かかります（特に初回）
- 曲の生成には30秒〜2分程度かかります
- Google Colabの無料版はメモリに制限があるため、多数の曲や長時間の生成は避けてください
- Colabのセッションは一定時間（約90分）で切断されることがあります

⚠️ **音楽生成に失敗する場合は、GPUを有効にしてください**：
1. 上部メニューの「ランタイム」→「ランタイムのタイプを変更」
2. 「ハードウェアアクセラレータ」で「GPU」を選択
3. 「保存」をクリック
4. 再度下の▶ボタンをクリック

In [ ]:
# ============== 環境とライブラリセットアップ ==============
# GPU設定を自動的に確認・推奨
import torch
if torch.cuda.is_available():
    gpu_info = torch.cuda.get_device_name(0)
    print(f"✅ GPUが利用可能です: {gpu_info}")
    print("⚡ ハードウェアアクセラレーションを使って高速生成できます")
    # GPUのタイプに基づいて推定パフォーマンスを表示
    if 'A100' in gpu_info:
        print("🔥 最高性能のGPUが割り当てられました！超高速処理が可能です")
    elif 'V100' in gpu_info or 'P100' in gpu_info:
        print("👍 高性能GPUが割り当てられました。高速処理が可能です")
    elif 'T4' in gpu_info:
        print("👌 標準的なGPUが割り当てられました。通常速度で処理できます")
    else:
        print("😊 GPUが割り当てられました。CPU処理より高速です")
else:
    print("⚠️ GPUが検出されませんでした。音楽生成に非常に時間がかかる可能性があります")
    print("💡 推奨: 上部メニューの「ランタイム」→「ランタイムのタイプを変更」→「GPU」を選択してください")
    user_input = input("GPUなしで続行しますか？ (y/n): ")
    if user_input.lower() != 'y':
        print("セットアップを中止します。GPUを有効にしてから再度実行してください。")
        import sys
        sys.exit()

print("\n🔄 必要なライブラリをインストールしています...（5〜10分かかります）")

# 必要なライブラリのインストール
!pip install -q torch torchaudio
!pip install -q gradio>=4.0.0 pydub>=0.25.1
!pip install -q git+https://github.com/facebookresearch/audiocraft.git

print("\n📥 アプリケーションコードをダウンロードしています...")

# GitHubからアプリケーションコードを直接ダウンロード
!mkdir -p phantom-track
!wget -q -O phantom-track/app.py https://raw.githubusercontent.com/Ryuto1991/phantom-track/main/app.py

In [ ]:
# ============== app.pyが正しくロードされているか確認 ==============
import os
import sys

# 現在のディレクトリパスを取得し、phantom-trackディレクトリへの絶対パスを作成
current_dir = os.getcwd()
phantom_track_dir = os.path.join(current_dir, 'phantom-track')

# app.pyの内容を表示（デバッグ用）
print("===== app.py内容の確認 =====")
app_path = os.path.join(phantom_track_dir, 'app.py')
if os.path.exists(app_path):
    with open(app_path, 'r') as f:
        app_content = f.read()
    # 内容が長いので先頭の10行と末尾の10行だけ表示
    app_lines = app_content.split('\n')
    print(f"ファイルサイズ: {len(app_content)} バイト, 行数: {len(app_lines)}")
    print("\n先頭10行:")
    print('\n'.join(app_lines[:10]))
    print("\n...")
    print("\n末尾10行:")
    print('\n'.join(app_lines[-10:]))
    
    # create_ui関数が存在するか確認
    if 'def create_ui(' in app_content:
        print("\n✅ create_ui関数が見つかりました")
    else:
        print("\n❌ create_ui関数が見つかりません！アプリが起動できない可能性があります")
else:
    print(f"❌ エラー: {app_path} が見つかりません")

In [ ]:
# ============== appモジュールからcreate_ui関数をインポートできるか確認 ==============
import os
import sys

# 現在のディレクトリパスを取得し、phantom-trackディレクトリへの絶対パスを作成
current_dir = os.getcwd()
phantom_track_dir = os.path.join(current_dir, 'phantom-track')

# app.pyをインポートできるかテスト
print("===== appモジュールのインポートテスト =====")
try:
    # phantom-trackディレクトリをPythonパスに追加
    sys.path.append(phantom_track_dir)
    # デバッグ用にシステムパスを表示
    print(f"システムパス: {sys.path}")
    
    # ディレクトリを移動してからインポートを試みる
    print(f"実行中のディレクトリ: {os.getcwd()}")
    orig_dir = os.getcwd()
    os.chdir(phantom_track_dir)
    print(f"移動後のディレクトリ: {os.getcwd()}")
    
    # appモジュールだけをインポート
    try:
        import app
        print("✅ appモジュールのインポートに成功しました")
        print(f"モジュール内容: {dir(app)}")
    except ImportError as e:
        print(f"❌ appモジュールのインポートに失敗: {e}")
    
    # create_ui関数をインポート
    try:
        from app import create_ui
        print("✅ create_ui関数のインポートに成功しました")
    except ImportError as e:
        print(f"❌ create_ui関数のインポートに失敗: {e}")
    
    # 元のディレクトリに戻る
    os.chdir(orig_dir)
except Exception as e:
    print(f"❌ エラー: {e}")

In [ ]:
# ============== アプリの起動 ==============
import os
import sys

print("\n🚀 Phantom Track Generatorを起動しています...")

# 現在のディレクトリパスを取得し、phantom-trackディレクトリへの絶対パスを作成
current_dir = os.getcwd()
phantom_track_dir = os.path.join(current_dir, 'phantom-track')

# app.pyを直接実行して、create_ui関数を呼び出す
try:
    # モデルのロードメッセージを表示
    print("\n🧠 AI音楽生成モデルを読み込んでいます...（初回は数分かかります）")
    print("⏳ しばらくお待ちください...")
    
    # ディレクトリを移動してから相対インポート
    os.chdir(phantom_track_dir)
    sys.path.append('.')
    from app import create_ui
    # 元のディレクトリに戻る
    os.chdir(current_dir)
    
    # アプリケーションを起動
    app = create_ui()
    public_url = app.launch(debug=False, share=True, quiet=True)
    
    print("\n✨ セットアップ完了！以下のURLをクリックしてアプリを開いてください:")
    print(f"🔗 {public_url}")
    print("\n📝 使い方:")
    print("1. 上記URLをクリックしてアプリを開く")
    print("2. 音楽ファイルをアップロードする")
    print("3. 生成指示コードを入力する")
    print("4. INITIATE GENERATIONボタンをクリック")
    print("\n⚠️ 注意: セッションは約90分でタイムアウトします")
except ImportError as e:
    print(f"❌ エラー: モジュールのインポートに失敗しました - {e}")
    print("\n💡 解決策を試しています...")
    try:
        # app.pyの内容を表示して問題を特定
        with open(os.path.join(phantom_track_dir, 'app.py'), 'r') as f:
            app_content = f.read()
        if 'def create_ui' not in app_content:
            print("❌ app.pyにcreate_ui関数が見つかりません")
        else:
            print("✅ app.pyにcreate_ui関数は存在します。インポート方法に問題がある可能性があります")
    except Exception as e2:
        print(f"❌ 追加確認でもエラーが発生: {e2}")
    sys.exit(1)
except Exception as e:
    print(f"❌ 予期せぬエラーが発生しました: {e}")
    print("\n💡 トラブルシューティング情報:")
    print("1. インターネット接続を確認してください")
    print("2. ブラウザを最新版に更新してください")
    print("3. GPUランタイムが有効になっているか確認してください")
    sys.exit(1)

In [ ]:
# ============== ランタイム再起動直後に一度だけ実行 ==============

print("🧹 step1 | 既存依存を削除")
!pip uninstall -y -q audiocraft torch torchaudio torchvision tensorboard numpy fastai gradio

print("⚙️ step2 | NumPy & TensorBoard を互換版に固定")
!pip install -q --no-deps numpy==1.26.4 tensorboard==2.15.1

print("🎵 step3 | PyTorch + Gradio + pydub + audiocraft をインストール")
!pip install -q --no-deps \
    torch==2.1.2+cu121 torchaudio==2.1.2+cu121 torchvision==0.16.0+cu121 \
    --index-url https://download.pytorch.org/whl/cu121
!pip install -q --no-deps gradio==4.19.0 pydub \
    "audiocraft@git+https://github.com/facebookresearch/audiocraft.git"

print("↘️ step4 | Phantom Track を取得＆パッチ")
!rm -rf phantom-track && git clone -q https://github.com/Ryuto1991/phantom-track.git

import pathlib, re, textwrap
app_py = pathlib.Path("phantom-track/app.py")
txt = app_py.read_text()

# ── (A) audiocraft import パス互換パッチ ─────────────────────
if "from audiocraft.models import MusicGen" in txt:
    txt = re.sub(
        r"from audiocraft\\.models import MusicGen",
        textwrap.dedent("""
            try:
                from audiocraft.models import MusicGen
            except ImportError:
                from audiocraft.models.musicgen import MusicGen
        """).strip(),
        txt
    )

# ── (B) gradio バージョン互換性パッチ ───────────────────────
# 古いGradio APIを最新版に合わせて修正
txt = re.sub(
    r"with gr\.Box\(elem_classes=\"([^\"]+)\"\):",
    r"with gr.Box(elem_id='\1'):",
    txt
)

# ── (C) デバッグ用コード追加 ───────────────────────────────
# エラー詳細ログを有効化
debug_code = """
# デバッグ用設定を追加
import logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger("phantom-track")
"""

if "import os" in txt:
    txt = txt.replace("import os", "import os\nimport sys\nimport traceback")

# デバッグ用のエラーキャッチを追加
txt = re.sub(
    r"def process_generation\(([^)]+)\):",
    r"def process_generation(\1):\n    logger.debug('process_generation開始')",
    txt
)

# デバッグログを追加
txt = re.sub(
    r"except Exception as e:",
    r"except Exception as e:\n        logger.error(f'エラー詳細: {str(e)}')\n        logger.error(traceback.format_exc())",
    txt
)

app_py.write_text(txt)

print("🔧 step5 | デバッグ設定と依存関係の修正")
# さらに必要な依存関係を追加
!pip install -q requests pillow

print("🚀 step6 | アプリ起動")
import sys
sys.path.append("phantom-track")
import app
ui = app.create_ui()
ui.launch(debug=True, share=True)